In [4]:
import numpy as np 
import scipy as sp

In [5]:
import numpy as np

# ===================================

get_skew = lambda a: np.array([[ 0, -a[2, 0], a[1, 0]], [a[2, 0], 0, -a[0, 0]], [-a[1, 0], a[0, 0], 0]])

def get_w_from_skew(skew):
    return np.array([[skew[2, 1]], [skew[0, 2]], [skew[1, 0]]])


# ===================================

get_linear_angular_velocity_from_twist = lambda twist: (twist[:3][:3], twist[3:][:3])

create_twist_from_linear_angular_velocity = lambda linear_velocity, angular_velocity: np.vstack((linear_velocity, angular_velocity))

def get_twist_skew(twist):
    w, v = get_linear_angular_velocity_from_twist(twist)
    w_skew = get_skew(w)
    twist_skew = np.zeros((4, 4))

    twist_skew[0:3, 0:3] = w_skew
    
    # ! twist_skew[:3, -1] returns a view of the last column of twist_skew but it is a row vector, v is a column vector
    # ! so we need to transpose v to make it a row vector and set it by reference
    twist_skew[:3, -1] = v.T

    return twist_skew


# ===================================

get_time_from_w_time = lambda w_t: np.linalg.norm(w_t)

# ===================================

# Define the rotation matrices for each axis

def get_x_rotation_matrix(theta, is_hybrid = False):
    
    if is_hybrid:
        return np.array([
            [1, 0, 0, 0],
            [0, np.cos(theta), -np.sin(theta), 0],
            [0, np.sin(theta), np.cos(theta), 0],
            [0, 0, 0, 1]
        ])
    else:
        return np.array([[1, 0, 0],
                        [0, np.cos(theta), -np.sin(theta)],
                        [0, np.sin(theta), np.cos(theta)]])
    
def get_y_rotation_matrix(theta, is_hybrid = False):
    if is_hybrid:
        return np.array([
        [np.cos(theta), 0, np.sin(theta), 0],
        [0, 1, 0, 0],
        [-np.sin(theta), 0, np.cos(theta), 0],
        [0, 0, 0, 1]
    ])
    else:
        return np.array([[np.cos(theta), 0, np.sin(theta)],
               [0, 1, 0],
               [-np.sin(theta), 0, np.cos(theta)]])
    
def get_z_rotation_matrix(theta, is_hybrid = False):
    if is_hybrid:
        return np.array([
            [np.cos(theta), -np.sin(theta), 0, 0],
            [np.sin(theta), np.cos(theta), 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])
    else:      
        return np.array([[np.cos(theta), -np.sin(theta), 0],
                [np.sin(theta), np.cos(theta), 0],
                [0, 0, 1]])

# 5.1

[![image.png](https://i.postimg.cc/wvb8dyZ2/image.png)](https://postimg.cc/2bvXQ6zL)

In [6]:

w_0_1 = w01 = np.array([[-2.53], [-0.90], [1.64]])

final_time = t = 8.00

Both General case and Special case give the same answer, the only thing that is different is the rigour

### Special case based on the info in the question T_s_b[0] = $I$

In [7]:
# T_s_b[t] = e^([V_s] * t) * T_s_b[0] 

# T_s_b[0] = I so

# T_s_b[t] = e^([V_s] * t)

w_s_skew = get_skew(w01)
V_s_skwq = np.eye(4)
V_s_skwq[0:3, 0:3] = w_s_skew * t
V_s_skwq[3, 3] = 0

ans = sp.linalg.expm(V_s_skwq)[0:3, 0:3]
ans

array([[ 0.99972697, -0.02030428, -0.01156379],
       [ 0.02065957,  0.99929078,  0.03148196],
       [ 0.01091637, -0.03171226,  0.99943742]])

### General case makes no assumptions

[![image.png](https://i.postimg.cc/Gh4MfSbZ/image.png)](https://postimg.cc/TpXqL0L9)

In [8]:
T_s_b = [0 for i in range(10)] 

t_inc = final_time / 10

curr_t = 0

curr_t += t_inc

T_s_b[0] = np.eye(4)

for i in range(1, 10):
    curr_t += t_inc # it is now the end of the interval

    w_s_skew = get_skew(w_0_1)
    V_s_skwq = np.eye(4)
    
    V_s_skwq[0:3, 0:3] = w_s_skew * curr_t
    T_s_b[i] = sp.linalg.expm(V_s_skwq) @ T_s_b[0]

ans = T_s_b[-1][0:3, 0:3]
ans


array([[ 0.99972697, -0.02030428, -0.01156379],
       [ 0.02065957,  0.99929078,  0.03148196],
       [ 0.01091637, -0.03171226,  0.99943742]])

# 5.2

[![image.png](https://i.postimg.cc/4NHF5ZPm/image.png)](https://postimg.cc/svycjktr)

[![image.png](https://i.postimg.cc/DZy0gMtN/image.png)](https://postimg.cc/HJRTWBW0)

In [18]:
R_0_1_after_motion = R = np.array([[-0.62306271, -0.78162272, -0.02930497], 
                                   [0.01270142, 0.02735061, -0.99954521], 
                                   [0.78206876, -0.62315156, -0.00711342]])

In [23]:
# * take the log of the matrix, it is defined differently than the log of a number
w_skew = sp.linalg.logm(R_0_1_after_motion)
w_skew

array([[-2.90679742e-10, -1.66036524e+00, -1.69600374e+00],
       [ 1.66036524e+00, -4.57894300e-11, -7.86770657e-01],
       [ 1.69600375e+00,  7.86770655e-01,  1.02830907e-08]])

In [20]:
w_t = get_w_from_skew(w_skew)

# ! We have w*time from the prior simplification
# ! we need to get time from w*time
# ! we know that norm of w is 1 (magnitude of w is 1) so norm(w*time) = time
theta = get_time_from_w_time(w_t)

theta

2.500449855782789

In [24]:
w = w_t / theta
w

array([[ 0.31465164],
       [-0.67827944],
       [ 0.66402661]])

# 5.3

[![image.png](https://i.postimg.cc/m25xV87g/image.png)](https://postimg.cc/JGZdz3Tf)

In [100]:
R_0_1 = R_1in0 = np.array([[0.98651331, -0.00862374, 0.16345371], [-0.02207429, 0.98247873, 0.18506290], [-0.16218573, -0.18617513, 0.96903798]])

R_0_2 = R_2in0 = np.array([[-0.58487875, -0.61794938, 0.52540976], [-0.70121753, 0.71079190, 0.05539711], [-0.40768961, -0.33602594, -0.84904402]])

In [101]:
# R_0_1 = A * R_0_2
# R_0_2^-1 * A = R_0_2^-1 * R_0_1
# A = R_0_2^-1 * R_0_1

rotation_matrix = R_0_1 @ R_0_2.T
# * take the log of the matrix, it is defined differently than the log of a number
w_skew = sp.linalg.logm(rotation_matrix)

In [102]:
w_t = get_w_from_skew(w_skew)

# ! We have w*time from the prior simplification
# ! we need to get time from w*time
# ! we know that norm of w is 1 (magnitude of w is 1) so norm(w*time) = time
theta = get_time_from_w_time(w_t)
theta

2.3860415193477187

In [103]:
w = w_t / theta
w

array([[ 0.37432592],
       [-0.91668301],
       [ 0.13990126]])

[![image.png](https://i.postimg.cc/2jLnV5T9/image.png)](https://postimg.cc/z35yQ8sT)

In [104]:
R_0_1 = R_1in0 = np.array([[0.72144568, -0.37900160, 0.57954630], [0.33814942, 0.92317263, 0.18277654], [-0.60429388, 0.06410991, 0.79417808]])

R_0_2 = R_2in0 = np.array([[-0.90665973, -0.37500351, -0.19323692], [0.03568395, -0.52458748, 0.85060839], [-0.42035080, 0.76431691, 0.48900395]])

In [105]:
# R_0_1 = A * R_0_2
# R_0_2^-1 * A = R_0_2^-1 * R_0_1
# A = R_0_2^-1 * R_0_1

rotation_matrix = R_0_1 @ R_0_2.T 
# * take the log of the matrix, it is defined differently than the log of a number
w_skew = sp.linalg.logm(rotation_matrix)

In [106]:
w_t = get_w_from_skew(w_skew) 

# ! We have w*time from the prior simplification
# ! we need to get time from w*time
# ! we know that norm of w is 1 (magnitude of w is 1) so norm(w*time) = time
theta = get_time_from_w_time(w_t)
theta

2.2455075509614613

In [107]:
w = w_t / theta
w =  R_0_2.T @ w
w

array([[ 0.38165294],
       [-0.45171891],
       [-0.80640626]])

# 5.4

In [108]:
import numpy as np

R_0_1 = R01 = np.array([[-0.186832403033, -0.947032489617, 0.261195552769], [-0.192194961728, 0.295975556200, 0.935659963244], [-0.963407883434, 0.124611130087, -0.237312697501]])

w_0 = w0 = np.array([[-2.590000000000], [0.070000000000], [-1.620000000000]])

final_time = t = 10.370000000000

p_1_1 =p1 = np.array([[-3.130000000000], [1.860000000000], [2.000000000000]])

In [109]:
# [[5.52683078305], [-3.71729696938], [0.379095390703]]

p_0_initial = R_0_1 @ p_1_1


w_skew = np.zeros((4, 4))

w_0_skew = get_skew(w_0)

linear_veloctiy_of_point = w_0_skew @ p_0_initial


linear_veloctiy_of_point

# w_skew[0:4, 3]



array([[ 5.09199886],
       [ 8.2410529 ],
       [-7.78481688]])

In [110]:
w_skew[0:3, 3] = [linear_veloctiy_of_point[0][0], linear_veloctiy_of_point[1][0], linear_veloctiy_of_point[2][0]]


# p_0_0_final = 

p_0_initial = np.array([p_0_initial[0], p_0_initial[1], p_0_initial[2], [1]])

ans = sp.linalg.expm(w_skew * final_time) @ p_0_initial
ans = R_0_1.T @ ans[0:3]
ans


array([[ 48.35409166],
       [-32.9128185 ],
       [114.91132465]])

# 5.5

[![image.png](https://i.postimg.cc/vBZqHsNX/image.png)](https://postimg.cc/t18t2LyV)

In [111]:
V_sbinb = np.array([[-7], [ 8], [-7], [ 7], [-7], [-4]])

In [112]:
# angular velocity of the body frame in the space frame
w_sbinb = V_sbinb[:3][:3]
w_sbinb


array([[-7],
       [ 8],
       [-7]])

In [113]:
# Translation velocity of the body frame in the space frame
v_sbinb = V_sbinb[3:][:3]
v_sbinb

array([[ 7],
       [-7],
       [-4]])

In [114]:
V_sbinb = np.array([[-5], [ 0], [-1], [-8], [-6], [ 1]])

In [115]:
# angular velocity of the body frame in the space frame
w_sbinb = V_sbinb[:3][:3]
w_sbinb


array([[-5],
       [ 0],
       [-1]])

In [116]:
# Translation velocity of the body frame in the space frame
v_sbinb = V_sbinb[3:][:3]
v_sbinb

array([[-8],
       [-6],
       [ 1]])

# 5.6

[![image.png](https://i.postimg.cc/Y0KxLL2r/image.png)](https://postimg.cc/cgh3VH82)

[![image.png](https://i.postimg.cc/Y0vw0V9K/image.png)](https://postimg.cc/hQRY32Tp)

In [117]:
V = np.array([[ 3], [ 4], [-2], [-8], [ 4], [-9]])

In [118]:
w, v = get_linear_angular_velocity_from_twist(V)
w_skew = get_skew(w)

In [119]:
twist_skew = np.zeros((4, 4))

twist_skew[0:3, 0:3] = w_skew
twist_skew[:3, -1] = v.T

twist_skew

array([[ 0.,  2.,  4., -8.],
       [-2.,  0., -3.,  4.],
       [-4.,  3.,  0., -9.],
       [ 0.,  0.,  0.,  0.]])

[![image.png](https://i.postimg.cc/4xNjxqF8/image.png)](https://postimg.cc/0bFVtWvm)

In [120]:
V = np.array([[ 2], [-3], [ 4], [-7], [ 6], [-5]])

In [121]:
get_twist_skew(V)

array([[ 0., -4., -3., -7.],
       [ 4.,  0., -2.,  6.],
       [ 3.,  2.,  0., -5.],
       [ 0.,  0.,  0.,  0.]])